In [1]:
'''
Author: sunflower-you m13119160579@163.com
Date: 2024-05-21 22:13:30
LastEditors: sunflower-you m13119160579@163.com
LastEditTime: 2024-05-21 22:53:03
FilePath: /python/project/register/register.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
from easydict import EasyDict

In [2]:
def _register_generic(module_dict, module_name, module):
    if isinstance(module_name, list):
        for module_name_item in module_name:
            assert module_name_item not in module_dict
            module_dict[module_name_item] = module
    else:
        assert module_name not in module_dict
        module_dict[module_name] = module

In [3]:
class Registry(EasyDict):
    """
        注册器：
    """

    def __init__(self, *args, **kwargs):
        super(Registry, self).__init__(*args, **kwargs)

    def register(self, module_name, module=None):
        # used as function call
        if module is not None:
            if isinstance(module, object):
                assert hasattr(module, "__call__")
            _register_generic(self, module_name, module)
            return

        # used as decorator
        def register_fn(fn):
            _register_generic(self, module_name, fn)
            return fn

        return register_fn


In [4]:
ACTIVATION_class = Registry()

In [5]:
from typing import Any
import torch

class Activation:
    def __init__(self) -> None:
        pass

    def __call__(self, x) -> Any:
        raise NotImplementedError()

class Silu(Activation):
    act_type = "silu"
    
    def __init__(self) -> None:
        super().__init__()

    def __call__(self, x) -> torch.Tensor:
        return x * torch.sigmoid(x)
    

class Relu(Activation):
    act_type = "relu"

    def __init__(self) -> None:
        super().__init__()

    def __call__(self, x) -> torch.Tensor:
        return torch.relu(x)

In [6]:
from enum import Enum, IntEnum

class ActivationType(str, Enum):
    Silu = "Silu"
    Relu = "Relu"

In [7]:
ACTIVATION_class.register(ActivationType.Silu, Silu)
ACTIVATION_class.register(ActivationType.Relu, Relu)

In [8]:
# test
input_tensor = torch.arange(-1, 5)

In [9]:
output = ACTIVATION_class["Relu"]()(input_tensor)
print(output)

tensor([0, 0, 1, 2, 3, 4])
